CreatedBy - Garvit Kumar Arya

Date - 26 August 2021

In [1]:
import subprocess

## Uploading *mlruns* directory to S3

In [5]:
s3_bucket_name = "garvit-mlops-sagemaker-runs-bucket"
mlruns_direc = "./mlruns/"

# output = subprocess.run(["aws", "s3", "sync", "{}".format(mlruns_direc), "s3://{}".format(s3_bucket_name)], stdout=subprocess.PIPE, encoding='utf-8')
output = subprocess.run(f"aws s3 sync {mlruns_direc} s3://{s3_bucket_name}")
print(output.stdout)
print("\nSaved to bucket: ", s3_bucket_name)



Saved to bucket:  garvit-mlops-sagemaker-runs-bucket


## Deploying 

In [18]:
import boto3
import mlflow.sagemaker as mfs
import json

app_name = "mlops-sagemaker"
execution_role_arn = "arn:aws:iam::113605498884:role/Sagemaker-S3-ECR-role"
image_ecr_url = "113605498884.dkr.ecr.us-east-2.amazonaws.com/mlflow-pyfunc:1.19.0"
region = "us-east-2"

s3_bucket_name = "garvit-mlops-sagemaker-runs-bucket"
experiment_id = "1"
run_id = "0a3bfd1192bd4f189a14d4522b902a01"
model_name = "log_reg_model"

# model_uri = f"s3://{s3_bucket_name}/{experiment_id}/{run_id}/{model_name}"
model_uri = "s3://garvit-mlops-sagemaker-runs-bucket/1/0a3bfd1192bd4f189a14d4522b902a01/artifacts/log_reg_model"
print(model_uri)

s3://garvit-mlops-sagemaker-runs-bucket/1/0a3bfd1192bd4f189a14d4522b902a01/artifacts/log_reg_model


In [19]:
mfs.deploy(app_name=app_name,
            model_uri=model_uri,
            execution_role_arn=execution_role_arn,
            region_name=region,
            image_url=image_ecr_url,
            mode=mfs.DEPLOYMENT_MODE_CREATE)

2021/08/26 17:11:18 INFO mlflow.sagemaker: Using the python_function flavor for deployment!
2021/08/26 17:11:21 INFO mlflow.sagemaker: No model data bucket specified, using the default bucket
2021/08/26 17:11:27 INFO mlflow.sagemaker: Default bucket `mlflow-sagemaker-us-east-2-113605498884` not found. Creating...
2021/08/26 17:11:32 INFO mlflow.sagemaker: Bucket creation response: {'ResponseMetadata': {'RequestId': 'HQDRJS6X97K8Q825', 'HostId': 'ODu+q92MUsKxBamc126MsNp/xDGQ/j9S/K3/IlwND2HSnBVK58fURqrIKr4ngiCLOCGGVXMzw8U=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'ODu+q92MUsKxBamc126MsNp/xDGQ/j9S/K3/IlwND2HSnBVK58fURqrIKr4ngiCLOCGGVXMzw8U=', 'x-amz-request-id': 'HQDRJS6X97K8Q825', 'date': 'Thu, 26 Aug 2021 11:41:33 GMT', 'location': 'http://mlflow-sagemaker-us-east-2-113605498884.s3.amazonaws.com/', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}, 'Location': 'http://mlflow-sagemaker-us-east-2-113605498884.s3.amazonaws.com/'}
2021/08/26 17:11:35 INFO mlflo

## Making Predictions

In [24]:
def query(input_json):
    client = boto3.session.Session().client("sagemaker-runtime", region)
    response = client.invoke_endpoint(EndpointName=app_name,
                                        Body=input_json,
                                        ContentType='application/json; format=pandas-split',)
    preds = response['Body'].read().decode("ascii")
    preds = json.loads(preds)
    return preds

In [25]:
import pandas as pd
import mlflow
import mlflow.sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
import numpy as np

data_path = "/home/garvit/Documents/work_area/Anaconda_Workspace/misc/MLOps with MLFlow/archive/creditcard.csv"
df = pd.read_csv(data_path)

normal = df[df.Class == 0].sample(frac=0.5, random_state=2020).reset_index(drop=True)
anomaly = df[df.Class == 1]
normal_train, normal_test = train_test_split(normal, test_size = 0.2, random_state = 2020)
anomaly_train, anomaly_test = train_test_split(anomaly, test_size = 0.2, random_state = 2020)

scaler = StandardScaler()
scaler.fit(pd.concat((normal, anomaly)).drop(["Time","Class"], axis=1))

scaled_selection = scaler.transform(df.iloc[:80].drop(["Time", "Class"], axis=1))
input_json = pd.DataFrame(scaled_selection).to_json(orient="split")
pd.DataFrame(query(input_json)).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
test = pd.concat((normal.iloc[:1900], anomaly.iloc[:100]))
true = test.Class
test = scaler.transform(test.drop(["Time", "Class"], axis=1))
preds = []


batch_size = 80
for f in range(25):
    print(f"Batch {f}", end=" - ")
    sample = pd.DataFrame(test[f*batch_size:(f+1)*batch_size]).to_json(orient="split")
    output = query(sample)
    resp = pd.DataFrame([output])
    preds = np.concatenate((preds, resp.values[0]))
    print("Completed")

eval_acc = accuracy_score(true, preds)
eval_auc = roc_auc_score(true, preds)
print("Eval Acc", eval_acc)
print("Eval AUC", eval_auc)


Batch 0 - Completed
Batch 1 - Completed
Batch 2 - Completed
Batch 3 - Completed
Batch 4 - Completed
Batch 5 - Completed
Batch 6 - Completed
Batch 7 - Completed
Batch 8 - Completed
Batch 9 - Completed
Batch 10 - Completed
Batch 11 - Completed
Batch 12 - Completed
Batch 13 - Completed
Batch 14 - Completed
Batch 15 - Completed
Batch 16 - Completed
Batch 17 - Completed
Batch 18 - Completed
Batch 19 - Completed
Batch 20 - Completed
Batch 21 - Completed
Batch 22 - Completed
Batch 23 - Completed
Batch 24 - Completed
Eval Acc 0.9915
Eval AUC 0.915


## Switching Models

In [29]:
new_run_id = "6e3b3920708345cd94e5cf1d22ceaee1"
new_model_uri = "s3://{}/{}/{}/artifacts/{}/".format(s3_bucket_name, experiment_id, new_run_id, model_name)
print(new_model_uri)

mfs.deploy( app_name=app_name,
            model_uri=new_model_uri,
            execution_role_arn=execution_role_arn,
            region_name=region,
            image_url=image_ecr_url,
            mode=mfs.DEPLOYMENT_MODE_REPLACE)

s3://garvit-mlops-sagemaker-runs-bucket/1/6e3b3920708345cd94e5cf1d22ceaee1/artifacts/log_reg_model/


2021/08/26 17:42:27 INFO mlflow.sagemaker: Using the python_function flavor for deployment!
2021/08/26 17:42:32 INFO mlflow.sagemaker: No model data bucket specified, using the default bucket
2021/08/26 17:42:37 INFO mlflow.sagemaker: Default bucket `mlflow-sagemaker-us-east-2-113605498884` already exists. Skipping creation.
2021/08/26 17:42:42 INFO mlflow.sagemaker: tag response: {'ResponseMetadata': {'RequestId': 'JS0K7CJHQSTJ6PSR', 'HostId': 'p/1FUf+IYFmFAQ0GVKfr0d1xNe3HfWWXgfCdfd7ZDsB8VdrfRZ8DXCTPxytAC/exvoGq4lYw2Uw=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'p/1FUf+IYFmFAQ0GVKfr0d1xNe3HfWWXgfCdfd7ZDsB8VdrfRZ8DXCTPxytAC/exvoGq4lYw2Uw=', 'x-amz-request-id': 'JS0K7CJHQSTJ6PSR', 'date': 'Thu, 26 Aug 2021 12:12:43 GMT', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}}
2021/08/26 17:42:44 INFO mlflow.sagemaker: Found active endpoint with arn: arn:aws:sagemaker:us-east-2:113605498884:endpoint/mlops-sagemaker. Updating...
2021/08/26 17:42:44 INFO mlflow.sage

## Removing Deployed Model

In [31]:
mfs.delete(app_name=app_name,region_name=region)

2021/08/26 18:12:51 INFO mlflow.sagemaker: Deleted endpoint with arn: arn:aws:sagemaker:us-east-2:113605498884:endpoint/mlops-sagemaker
2021/08/26 18:12:51 INFO mlflow.sagemaker: Waiting for the delete operation to complete...
2021/08/26 18:12:51 INFO mlflow.sagemaker: Deletion is still in progress. Current endpoint status: Deleting
2021/08/26 18:12:57 INFO mlflow.sagemaker: The deletion operation completed successfully with message: "The SageMaker endpoint was deleted successfully."
2021/08/26 18:12:57 INFO mlflow.sagemaker: Cleaning up unused resources...
2021/08/26 18:12:57 INFO mlflow.sagemaker: Deleted associated endpoint configuration with arn: arn:aws:sagemaker:us-east-2:113605498884:endpoint-config/mlops-sagemaker-config-vt1lmj67sdkr1blffjf4yq
2021/08/26 18:12:59 INFO mlflow.sagemaker: Deleted associated model with arn: arn:aws:sagemaker:us-east-2:113605498884:model/mlops-sagemaker-model-tudehbplru2zke5tfyxq9g
